In [ ]:
import sys
if '/Users/nehiljain/code/find-your-mate-ai/src' not in sys.path:
    sys.path.append('/Users/nehiljain/code/find-your-mate-ai/src')


import nest_asyncio
nest_asyncio.apply()
from find_your_mate_ai.config import settings
from find_your_mate_ai.data_ingestion import *
import pandas as pd
openai.api_key = settings.OPENAI_API_KEY
logging.info("OpenAI API key configured")


# nodes = load_nodes_from_mongodb(settings.MONGO_URI)


In [ ]:
# Import necessary modules and functions
from find_your_mate_ai.data_ingestion import ingest_profiles_data, create_pinecone_index, create_auto_retriever, MongoConnection, OpenAI
from llama_index.core.llms.llm import LLM
from pprint import pprint
from dynaconf import settings
# Print all settings using the test environment for visual inspection
with settings.using_env("test"):
    pprint(settings.to_dict())
    mongo_conn = MongoConnection(
        mongo_uri=settings.MONGO_URI,
        db_name=settings.MONGODB_NAME,
        collection_name="test_notebook_collection",
    )
    nodes = ingest_profiles_data(
        source_data_path=settings.SOURCE_DATA_PATH,
        output_data_path=settings.OUTPUT_DATA_PATH,
        openai_api_key=settings.OPENAI_API_KEY,
        mongo_conn=mongo_conn,
    )
    for node in nodes:
        print(node.metadata)
        print("-" * 100)
    # Create Pinecone index
    pinecone_index, pinecone_instance = create_pinecone_index(
        index_name="test-notebook-index",
        pinecone_api_key=settings.PINECONE_API_KEY
    )



In [ ]:
# Create a retriever
# Index profiles data
vector_store_index = index_profiles_data(
    pinecone_index=pinecone_index
)
retriever2 = create_auto_retriever(
    vector_store_index=vector_store_index,
    llm=OpenAI(model="gpt-4-turbo")
)
from llama_index.core.response.notebook_utils import display_source_node

filtered_nodes = retriever2.retrieve(
    "Who are some founders that are 30 or below"
)
for node in filtered_nodes:
    print(node.metadata['file_name'])
    display_source_node(node, source_length=1000)
    print("-"*100)
    print("-"*100)

